In [136]:
import pandas as pd

data=pd.read_csv('heart_disease_uci.csv')

In [137]:
data.head()

,id,age,sex,dataset,cp,trestbps,chol,fbs,restecg,thalch,exang,oldpeak,slope,ca,thal,num
0,1,63,Male,Cleveland,typical angina,145.0,233.0,True,lv hypertrophy,150.0,False,2.3,downsloping,0.0,fixed defect,0
1,2,67,Male,Cleveland,asymptomatic,160.0,286.0,False,lv hypertrophy,108.0,True,1.5,flat,3.0,normal,2
2,3,67,Male,Cleveland,asymptomatic,120.0,229.0,False,lv hypertrophy,129.0,True,2.6,flat,2.0,reversable defect,1
3,4,37,Male,Cleveland,non-anginal,130.0,250.0,False,normal,187.0,False,3.5,downsloping,0.0,normal,0
4,5,41,Female,Cleveland,atypical angina,130.0,204.0,False,lv hypertrophy,172.0,False,1.4,upsloping,0.0,normal,0


In [139]:
# Assuming 'data' is a DataFrame or a structure like that
categorical_features = data.select_dtypes(include=['object']).columns.tolist()
numerical_features = data.select_dtypes(exclude=['object']).columns.tolist()
if 'id' in numerical_features:
    numerical_features.remove('id')
    numerical_features.remove('num')
    
print("categorical_features")
print(categorical_features)
print("\nnumerical_features")
print(numerical_features)

categorical_features
['sex', 'dataset', 'cp', 'fbs', 'restecg', 'exang', 'slope', 'thal']

numerical_features
['age', 'trestbps', 'chol', 'thalch', 'oldpeak', 'ca']


In [140]:
from sklearn.impute import SimpleImputer

imputer=SimpleImputer(strategy='mean')

num_data=data.drop(categorical_features,axis=1)
imputer.fit(num_data)
imputer.statistics_




array([460.5       ,  53.51086957, 132.13240418, 199.13033708,
       137.54566474,   0.87878788,   0.6763754 ,   0.99565217])

In [141]:
# handling the missing values
data_imputed=imputer.transform(num_data)

imputed_df=pd.DataFrame(data_imputed,columns=num_data.columns)
data_1=pd.concat([imputed_df,data[categorical_features]],axis=1)

data_1.dropna(subset=categorical_features,inplace=True)  #inplace is used to ensure that new dataset is not created it just alter the original dataset
data_1.head()

# to check if rows are removed when categories value(s) are missing
# missing_categorical = data[categorical_features].isnull().sum()

# # Print or view the count of missing values in categorical columns
# print(missing_categorical)


target = data_1.iloc[:, 7]  # Assuming the target column is at index 7

# Remove the target column from the DataFrame to keep only features
data_1 = data_1.drop(columns=data_1.columns[7])


# to ensure that the target and data have the same number of rows as some rows where removed in because of category
# len(data_1)
# len(target)



In [144]:
# normalising

from sklearn.preprocessing import StandardScaler
scaler=StandardScaler()
data_1[numerical_features]=scaler.fit_transform(data_1[numerical_features])
data_1.head()

,id,age,trestbps,chol,thalch,oldpeak,ca,sex,dataset,cp,fbs,restecg,exang,slope,thal
0,1.0,0.912088,0.719280,0.185953,0.247221,1.151950,-0.802174,Male,Cleveland,typical angina,True,lv hypertrophy,False,downsloping,fixed defect
1,2.0,1.355613,1.558087,0.730148,-1.364202,0.434753,2.773361,Male,Cleveland,asymptomatic,False,lv hypertrophy,True,flat,normal
2,3.0,1.355613,-0.678732,0.144881,-0.558491,1.420899,1.581516,Male,Cleveland,asymptomatic,False,lv hypertrophy,True,flat,reversable defect
3,4.0,-1.970826,-0.119527,0.360506,1.666808,2.227746,-0.802174,Male,Cleveland,non-anginal,False,normal,False,downsloping,normal
4,5.0,-1.527300,-0.119527,-0.111815,1.091300,0.345103,-0.802174,Female,Cleveland,atypical angina,False,lv hypertrophy,False,upsloping,normal


In [147]:
data_1['fbs'].value_counts()  #to see the categories in the car=tegorical_features
# 'sex', 'dataset', 'cp', 'fbs', 'restecg', 'exang', 'slope', 'thal'


fbs
False    320
True      52
Name: count, dtype: int64

In [148]:
binary_cat=['sex','fbs','exang']
nominal_cat=['dataset']
hot_categorical_features=binary_cat+nominal_cat
# selecting these from the data
hot_data=data_1[hot_categorical_features]

from sklearn.preprocessing import OneHotEncoder
hot_encoder=OneHotEncoder(sparse_output=False,drop='first')
encoded_hot_data=hot_encoder.fit_transform(hot_data)
encoded_hot_columns=hot_encoder.get_feature_names_out(hot_categorical_features)
encoded_hot_df=pd.DataFrame(encoded_hot_data,columns=encoded_hot_columns)
data_1.drop(hot_categorical_features, axis=1, inplace=True)  # Drop original categorical columns
data_1 = pd.concat([data_1, encoded_hot_df], axis=1)






ordinal_cat=['cp','restecg','slope','thal']


In [149]:
data_1.head()

,id,age,trestbps,chol,thalch,oldpeak,ca,cp,restecg,slope,thal,sex_Male,fbs_True,exang_True,dataset_Hungary,dataset_Switzerland,dataset_VA Long Beach
0,1.0,0.912088,0.719280,0.185953,0.247221,1.151950,-0.802174,typical angina,lv hypertrophy,downsloping,fixed defect,1.0,1.0,0.0,0.0,0.0,0.0
1,2.0,1.355613,1.558087,0.730148,-1.364202,0.434753,2.773361,asymptomatic,lv hypertrophy,flat,normal,1.0,0.0,1.0,0.0,0.0,0.0
2,3.0,1.355613,-0.678732,0.144881,-0.558491,1.420899,1.581516,asymptomatic,lv hypertrophy,flat,reversable defect,1.0,0.0,1.0,0.0,0.0,0.0
3,4.0,-1.970826,-0.119527,0.360506,1.666808,2.227746,-0.802174,non-anginal,normal,downsloping,normal,1.0,0.0,0.0,0.0,0.0,0.0
4,5.0,-1.527300,-0.119527,-0.111815,1.091300,0.345103,-0.802174,atypical angina,lv hypertrophy,upsloping,normal,0.0,0.0,0.0,0.0,0.0,0.0


In [150]:
from sklearn.preprocessing import LabelEncoder

# Assuming 'data_1' is your DataFrame containing ordinal categorical features

ordinal_cat = ['cp', 'restecg', 'slope', 'thal']

# Initialize LabelEncoder
label_encoder = LabelEncoder()

# Apply label encoding to each ordinal categorical column
for feature in ordinal_cat:
    # Fill NaN values with a placeholder (if needed)
    data_1[feature].fillna('NaN', inplace=True)  # Replace NaN values if present
    
    # Fit and transform the ordinal categorical feature
    data_1[feature] = label_encoder.fit_transform(data_1[feature])

In [151]:
data_1.head()

,id,age,trestbps,chol,thalch,oldpeak,ca,cp,restecg,slope,thal,sex_Male,fbs_True,exang_True,dataset_Hungary,dataset_Switzerland,dataset_VA Long Beach
0,1.0,0.912088,0.719280,0.185953,0.247221,1.151950,-0.802174,4,1,1,1,1.0,1.0,0.0,0.0,0.0,0.0
1,2.0,1.355613,1.558087,0.730148,-1.364202,0.434753,2.773361,1,1,2,2,1.0,0.0,1.0,0.0,0.0,0.0
2,3.0,1.355613,-0.678732,0.144881,-0.558491,1.420899,1.581516,1,1,2,3,1.0,0.0,1.0,0.0,0.0,0.0
3,4.0,-1.970826,-0.119527,0.360506,1.666808,2.227746,-0.802174,3,2,1,2,1.0,0.0,0.0,0.0,0.0,0.0
4,5.0,-1.527300,-0.119527,-0.111815,1.091300,0.345103,-0.802174,2,1,3,2,0.0,0.0,0.0,0.0,0.0,0.0
